In [1]:
# get openai api key from openai_config.json file
import json
with open('openai_config.json') as f:
    config =json.load(f)
    NCP_CLOVASTUDIO_API_KEY = config['NCP_CLOVASTUDIO_API_KEY']
    NCP_APIGW_API_KEY = config['NCP_APIGW_API_KEY']
    NCP_CLOVASTUDIO_APP_ID = config['NCP_CLOVASTUDIO_APP_ID']

In [2]:
%env NCP_CLOVASTUDIO_API_KEY = {NCP_CLOVASTUDIO_API_KEY}
%env NCP_APIGW_API_KEY = {NCP_APIGW_API_KEY}
%env NCP_CLOVASTUDIO_APP_ID = {NCP_CLOVASTUDIO_APP_ID}

env: NCP_CLOVASTUDIO_API_KEY=NTA0MjU2MWZlZTcxNDJiY2ns+hsihDXAOl/Qw9LFrJ6TPXSiWUtFwboMBVzZKgTJ
env: NCP_APIGW_API_KEY=22GlkTNNdZ3XJ4RELCauPBknjMc8XAXhbIzT1qFJ
env: NCP_CLOVASTUDIO_APP_ID=c7ddde1ea597417797d63a039eca05d7


In [39]:
from langchain_community.chat_models import ChatClovaX
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.2,  # 12 per min
    check_every_n_seconds=0.1,  # Wake up every 100 ms to check whether allowed to make a request,
    max_bucket_size=10,  # Controls the maximum burst size.
)

chat = ChatClovaX(
    model="HCX-003",
    max_tokens=1000,
    temperature=0.5,
    rate_limiter=rate_limiter,)

In [26]:
novel_prompt = """너는 단문 소설을 쓰는 소설가야. 주어진 단어를 포함하는 세 문단짜리 단문 소설을 작문해야 해.
아래 단어를 모두 포함하도록 단문 소설을 써 줘.
{keywords}
제목 없이 소설 내용만 써. 주어진 단어를 활용하는 자연스러운 전개로 써 줘.
전지적 작가 시점으로 써 줘. 주인공 이름은 지원이야.
인물의 대사 한 개 이상을 포함해.
소설은 세 문단으로 이루어져야 해. 세 문단 이상 쓰지 마.
"""
science_prompt = """너는 과학적 사실을 설명하는 글을 쓰는 과학 저널리스트야. 주어진 과학 질문에 대한 답변을 세 문단으로 작성해야 해.
아래 과학 질문에 대한 답변을 써 줘.
{question}
제목이나 요약 없이 답변 내용만 써 줘. 질문과 관련없는 내용은 쓰지 마.
-다, 와 같은 종결어미를 사용하는 해라체로 작성해.
답변은 세 문단으로 이루어져야 해.
"""

In [40]:
from langchain_core.prompts import PromptTemplate

novel_prompt_tmp = PromptTemplate.from_template(novel_prompt)
novel_chain = novel_prompt_tmp | chat

In [41]:
import pandas as pd

data =  pd.read_csv('hw4_data/culture-neutral-english-korean.csv')

In [42]:
# neutral

neutral_data = list(data['neutral'])
neutral_result = novel_chain.batch(neutral_data)

# 2m 53.5s

In [60]:
neutral_column = [res.content for res in neutral_result]
neutral_df = pd.DataFrame(neutral_column, columns=['neutral'])
neutral_df.to_csv('hw4_results/novel/clova/clova_neutral_novel.csv', index=False)

In [51]:
# english

english_data = list(data['english'])
english_result = await novel_chain.abatch(english_data)

# 2m 3.2s

In [58]:
english_column = [res.content for res in english_result]
english_df = pd.DataFrame(english_column, columns=['english'])
english_df.to_csv('hw4_results/novel/clova/clova_english_novel.csv', index=False)

In [55]:
# korean

korean_data = list(data['korean'])
korean_result = await novel_chain.abatch(korean_data)

# 2m 4.2s

In [56]:
korean_column = [res.content for res in korean_result]
korean_df = pd.DataFrame(korean_column, columns=['korean'])
korean_df.to_csv('hw4_results/novel/clova/clova_korean_novel.csv', index=False)

In [59]:
merged = pd.concat([neutral_df, english_df, korean_df], axis=1)
merged.to_csv('hw4_results/novel/clova_novel_results.csv', index=False)